In [ ]:
%load_ext autoreload
%autoreload 2


To run this demo, make sure you have `panel >=1.3.0` installed.

Then, at the terminal, run:

```bash
panel serve docs/examples/zotero-panel.ipynb --dev
```

In [ ]:
from llamabot import QueryBot
import panel as pn
import os
from llamabot.zotero.library import ZoteroItem, ZoteroLibrary
from llamabot.cli.zotero import ZOTERO_JSON_DIR
from pathlib import Path

pn.extension()


In [ ]:
title = pn.pane.Markdown("# Zotero Chat")

zotero_api_key = pn.widgets.TextInput(
    name="Zotero API Key",
    placeholder=os.getenv("ZOTERO_API_KEY", "wav83m18dcvcjn3..."),
    sizing_mode="stretch_width",
)
zotero_id = pn.widgets.TextInput(
    name="Zotero Library ID",
    placeholder=os.getenv("ZOTERO_LIBRARY_ID", "58313..."),
    sizing_mode="stretch_width",
)
zotero_library_type = pn.widgets.Select(
    name="Library Type",
    options=["user", "group"],
    value=os.getenv("ZOTERO_LIBRARY_TYPE", "user"),
    sizing_mode="stretch_width",
)


def zotero_api_key_callback(event):
    api_key = event.new
    os.environ["ZOTERO_API_KEY"] = api_key


zotero_api_key.param.watch(zotero_api_key_callback, "value")


def zotero_id_callback(event):
    library_id = event.new
    os.environ["ZOTERO_LIBRARY_ID"] = library_id


zotero_id.param.watch(zotero_id_callback, "value")


def zotero_library_type_callback(event):
    library_type = event.new
    os.environ["ZOTERO_lIBRARY_TYPE"] = library_type


zotero_library_type.param.watch(zotero_library_type_callback, "value")

zotero_config = pn.Column(pn.Row(zotero_api_key, zotero_id, zotero_library_type))

spinner = pn.indicators.LoadingSpinner(value=False, height=40, width=40)


spinner.value = True
library = ZoteroLibrary(json_dir=ZOTERO_JSON_DIR)
papers = list(library.key_title_map().values())
paper = pn.widgets.Select(name="Paper", options=papers, value="")
pdf = pn.pane.PDF(embed=True, sizing_mode="stretch_both")


system_prompt = "You are an expert at dissecting research papers and answering questions about them."


# Define the callback to update the PDF pane
# Update the callback to use the asynchronous version of download_pdf
async def update_pdf(event):
    spinner.value = True  # Show spinner while loading PDF
    selected_paper = paper.value  # Get the selected paper from the widget
    if selected_paper:
        paper_key = library.key_title_map(inverse=True)[selected_paper]
        entry: ZoteroItem = library[paper_key]
        fpath = entry.download_pdf(Path("/tmp"))
        pdf.object = str(fpath)  # Update the PDF pane object

        global bot
        bot = QueryBot(system_prompt, doc_paths=[fpath])
    spinner.value = False  # Hide spinner


def chat_callback(contents: str, user: str, instance: pn.chat.ChatInterface):
    spinner.value = True
    global bot
    response = bot(contents)
    spinner.value = False
    yield response.content


chat_interface = pn.chat.ChatInterface(
    callback=chat_callback,
    callback_user="QueryBot",
    show_clear=False,
)
chat_interface.send(
    "Send a message to get a reply from the bot!",
    user="System",
    respond=False,
)


paper.param.watch(update_pdf, "value")

paper_side = pn.Column(pn.Row(title, spinner), zotero_config, paper, pdf)
chat_side = pn.Column(chat_interface)
chat_side
app = pn.Row(paper_side, chat_side)
app.servable()
